In [4]:
from konlpy.tag import Mecab 

mec = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

In [2]:
import pandas as pd
df = pd.read_csv("../../outputs/week1/df3_2_artsCleanEtc.csv")

In [19]:
from tqdm import tqdm

#초기 구조화: 필수 -> 이후 case로 분기
series_mec = df['art_title'].map(lambda x : mec.pos(x, flatten=False, join=False))

df_mec = pd.DataFrame(columns=['art_id','token'])

for i in tqdm( range(len(series_mec)), desc="초기 구조화 진행중"):
    for j in range(len(series_mec[i])):
        df_mec = df_mec.append({'art_id':i, 'token': series_mec[i][j]}, ignore_index=True)
     
# 저장하지 말고 그대로가야댐 utf변환때문에   
# # 저장주의!!!
# df_mec.to_csv('df_mec_title_all_2.csv',encoding='utf-8-sig', index=False)


초기 구조화 진행중: 100%|██████████| 15024/15024 [03:30<00:00, 71.26it/s]


### -> outputs/week3/df_mec_title_all.csv 저장됨
<br>

## Compound 로직 
#### input: df_mec_title_all

---

#### case
1. 명사간 -> comp_NNG_NNG
2. VA + EC -> comp_VA_EC
3. VA + EF
4. SN + NR + NNBC
5. SN + NR
6. VCN + EC
7. NNG + XSN + JKB
8. NNG + JKB
9. NNG + XSN
10. MM + NNBC
11. MM + NNG
12. VV + (EP+EF)
13. SN + NNBC
14. VA + ETM
15. XSV + EC
16. SL + NNG
17. ('1', 'SN'), ('년', 'NNBC'), ('차', 'NNG'), 

In [25]:
import os
import pandas as pd

,art_id,token
0,0,"[('erp', 'SL')]"
1,0,"[('개발', 'NNG')]"
2,0,"[('유지', 'NNG'), ('보수', 'NNG')]"
3,0,"[('하', 'VV'), ('고', 'EC'), ('있', 'VX'), ('는데',..."
4,0,"[('프로그램', 'NNG')]"


In [ ]:
# # (CASE1) 복합명사 처리x 기본 형태소 단위 그대로 넣은 경우 비교용

# df_res_basic = pd.DataFrame(columns=['art_id','token','type'])

# for i in range(len(df_mec_title)):
#     for j in range(len(df_mec_title.loc[i][1])):
#         df_res_basic = df_res_basic.append({'art_id':df_mec_title.loc[i][0], 'token':df_mec_title.loc[i][1][j][0], 'type':df_mec_title.loc[i][1][j][1]},ignore_index=True)
               
# print(df_res_basic)

# # df_res_basic.to_csv('df_res_basic.csv', encoding='utf-8-sig', index=False)

# CASE 일괄작업 로직 

#### ( input : df_mec : 저장된거 불러오면 안됨, 무조건 위에서 돌린거 그대로)

In [ ]:
from tqdm import tqdm

# sample
# df_mec_sample = df_mec.loc[0: 3000]

df_res_comps = pd.DataFrame(columns=['art_id','token','type'])
# df_res_comps.append({'art_id':, 'token','type'})

for i in tqdm(range(len(df_mec)), desc="TYPE 일괄조합 진행중"):
    # 3개 조합 케이스
    if len(df_mec.loc[i][1]) == 3 :
        # (1) 명사조합처리
        if (df_mec.loc[i][1][0][1] == 'NNG' or df_mec.loc[i][1][0][1] == 'NNP') and (df_mec.loc[i][1][1][1] == 'NNG' or df_mec.loc[i][1][1][1] == 'NNP') and (df_mec.loc[i][1][2][1] == 'NNG' or df_mec.loc[i][1][2][1] == 'NNP'):
                df_res_comps = df_res_comps.append({'art_id':df_mec.loc[i][0],'token': "".join([df_mec.loc[i][1][0][0], df_mec.loc[i][1][1][0], df_mec.loc[i][1][2][0] ]) ,'type':'comp_Nouns3'},ignore_index=True)
                continue
        # (2) 1 년 차 ( SN + NNBC + NNG ) 처리
        elif (df_mec.loc[i][1][0][1] == 'SN') and (df_mec.loc[i][1][1][1] == 'NNBC') and (df_mec.loc[i][1][2][1] == 'NNG'):
                df_res_comps = df_res_comps.append({'art_id':df_mec.loc[i][0],'token': "".join([df_mec.loc[i][1][0][0], df_mec.loc[i][1][1][0], df_mec.loc[i][1][2][0] ]) ,'type':'comp_SN_NNBC_NNG'},ignore_index=True)
                continue
        # (3) N 만 원 ( SN + NNBC + NNG ) 처리
        elif (df_mec.loc[i][1][0][1] == 'SN') and (df_mec.loc[i][1][1][1] == 'NR') and (df_mec.loc[i][1][2][1] == 'NNBC'):
                df_res_comps = df_res_comps.append({'art_id':df_mec.loc[i][0],'token': "".join([df_mec.loc[i][1][0][0], df_mec.loc[i][1][1][0], df_mec.loc[i][1][2][0] ]) ,'type':'comp_SN_NR_NNBC'},ignore_index=True)
                continue
        
        # (ex) 예외처리
        df_res_comps = df_res_comps.append({'art_id':df_mec.loc[i][0], 'token':df_mec.loc[i][1][0][0] ,'type':df_mec.loc[i][1][0][1]}, ignore_index=True)
        df_res_comps = df_res_comps.append({'art_id':df_mec.loc[i][0], 'token':df_mec.loc[i][1][1][0] ,'type':df_mec.loc[i][1][1][1]}, ignore_index=True)
        df_res_comps = df_res_comps.append({'art_id':df_mec.loc[i][0], 'token':df_mec.loc[i][1][2][0] ,'type':df_mec.loc[i][1][2][1]}, ignore_index=True)
    
    # 2개 조합 케이스
    elif len(df_mec.loc[i][1]) == 2 :
        # (1) 명사조합처리
        if (df_mec.loc[i][1][0][1] == 'NNG' or df_mec.loc[i][1][0][1] == 'NNP') and (df_mec.loc[i][1][1][1] == 'NNG' or df_mec.loc[i][1][1][1] == 'NNP') :
            df_res_comps = df_res_comps.append({'art_id':df_mec.loc[i][0],'token': "".join([df_mec.loc[i][1][0][0], df_mec.loc[i][1][1][0] ]) ,'type':'comp_Nouns2'},ignore_index=True)
            continue
        
        # (2) xx살 (SN + NNBC) 처리
        elif (df_mec.loc[i][1][0][1] == 'SN') and (df_mec.loc[i][1][1][1] == 'NNBC') :
            df_res_comps = df_res_comps.append({'art_id':df_mec.loc[i][0],'token': "".join([df_mec.loc[i][1][0][0], df_mec.loc[i][1][1][0] ]) ,'type':'comp_SN_NNBC'},ignore_index=True)
            continue
        
        # (3) 이 직 / 한 달 ( MM + NNG/NNP ) 처리
        elif (df_mec.loc[i][1][0][1] == 'MM') and (df_mec.loc[i][1][1][1] == 'NNG') :
            df_res_comps = df_res_comps.append({'art_id':df_mec.loc[i][0],'token': "".join([df_mec.loc[i][1][0][0], df_mec.loc[i][1][1][0] ]) ,'type':'comp_MM_NNG'},ignore_index=True)
            continue        
        
        # (ex) 예외처리
        df_res_comps = df_res_comps.append({'art_id':df_mec.loc[i][0], 'token':df_mec.loc[i][1][0][0] ,'type':df_mec.loc[i][1][0][1]}, ignore_index=True)
        df_res_comps = df_res_comps.append({'art_id':df_mec.loc[i][0], 'token':df_mec.loc[i][1][1][0] ,'type':df_mec.loc[i][1][1][1]}, ignore_index=True)
    
    # 1개 이하 처리
    elif len(df_mec.loc[i][1]) <= 1 :
        df_res_comps = df_res_comps.append({'art_id':df_mec.loc[i][0],'token':df_mec.loc[i][1][0][0],'type':df_mec.loc[i][1][0][1]},ignore_index=True)
        continue

    
pd.set_option('display.max_rows', None)
df_res_comps.head(100)

In [21]:
# 저장
df_res_comps.to_csv('./outputs/df0_1_res_totalComps_title.csv', encoding='utf-8-sig', index=False)

In [ ]:
# # sample ###############################백업본
# df_mec_sample = df_mec.loc[: 100]
# df_mec_sample.loc[3][1][0]

# df_res_comps = pd.DataFrame(columns=['art_id','token','type'])
# # df_res_comps.append({'art_id':, 'token','type'})

# for i in range(len(df_mec_sample)):
#     if len(df_mec_sample.loc[i][1]) <= 1 :
#         df_res_comps = df_res_comps.append({'art_id':df_mec_sample.loc[i][0],'token':df_mec_sample.loc[i][1][0][0],'type':df_mec_sample.loc[i][1][0][1]},ignore_index=True)
#         continue
#     for j in range(1,len(df_mec_sample.loc[i][1])):
#         if (df_mec_sample.loc[i][1][j-1][1] == 'NNG' or df_mec_sample.loc[i][1][j-1][1] == 'NNP') and (df_mec_sample.loc[i][1][j][1] == 'NNG' or df_mec_sample.loc[i][1][j][1] == 'NNP') :
#             df_res_comps = df_res_comps.append({'art_id':df_mec_sample.loc[i][0],'token': "".join([df_mec_sample.loc[i][1][j-1][0], df_mec_sample.loc[i][1][j][0] ]) ,'type':'comp_Nouns'},ignore_index=True)
#             continue
#         df_res_comps = df_res_comps.append({'art_id':df_mec_sample.loc[i][0], 'token':df_mec_sample.loc[i][1][j][0] ,'type':df_mec_sample.loc[i][1][j][1]}, ignore_index=True)

# print(df_res_comps)

In [4]:
# # (CASE3) mecab 배열상 type 연속성 기반 복합명사화 : NNG,NNP-NNG,NNP 혼재 CASE 
#     # 문제: 3개 연달아 등장하는 명사 해결법
#     # 문제: 연

# df_res_NNG_NNP = pd.DataFrame(columns=['art_id','token','type'])
# # df_res_NNG_NNP.append({'art_id':, 'token','type'})

# for i in range(len(df_mec)):
#     if len(df_mec.loc[i][1]) <= 1 :
#         df_res_NNG_NNP = df_res_NNG_NNP.append({'art_id':df_mec.loc[i][0],'token':df_mec.loc[i][1][0][0],'type':df_mec.loc[i][1][0][1]},ignore_index=True)
#         continue
#     for j in range(1,len(df_mec.loc[i][1])):
#         if (df_mec.loc[i][1][j-1][1] == 'NNG' or df_mec.loc[i][1][j-1][1] == 'NNP') and (df_mec.loc[i][1][j][1] == 'NNG' or df_mec.loc[i][1][j][1] == 'NNP') :
#             df_res_NNG_NNP = df_res_NNG_NNP.append({'art_id':df_mec.loc[i][0],'token': "".join([df_mec.loc[i][1][j-1][0], df_mec.loc[i][1][j][0] ]) ,'type':'comp_NNG_NNP'},ignore_index=True)
#             continue
#         df_res_NNG_NNP = df_res_NNG_NNP.append({'art_id':df_mec.loc[i][0], 'token':df_mec.loc[i][1][j][0] ,'type':df_mec.loc[i][1][j][1]}, ignore_index=True)


# df_res_NNG_NNP.to_csv('df_res_NNG_NNP_test1.csv', encoding='utf-8-sig', index=False)

# print(df_res_NNG_NNP)

     art_id token          type
0         0   erp            SL
1         0  프로그램           NNG
2         0    오류           NNG
3         0  장애조치  comp_NNG_NNP
4         0    방법           NNG
...     ...   ...           ...
6578   1000    고수           NNP
6579   1000     님           XSN
6580   1000     들           XSN
6581   1000     주            VX
6582   1000    세요         EP+EF

[6583 rows x 3 columns]
